In [1]:
import LinearAlgebra as la
using BenchmarkTools: @btime
include("hoSystem.jl");

In [2]:
l_0 = 10  # number of basis functions
ω = 0.25  # strength of harmonic oscillator potential
a = 0.25  # shielding term in coloumb interaction
grid = [x for x in range(-10, stop = 10, length = 2001)]
n = 2     # number of particles

@time system = System(n, HOBasis(l_0, ω, a), grid);

  4.068439 seconds (5.05 M allocations: 298.618 MiB, 4.59% gc time, 8.17% compilation time)


In [5]:
# Stand-in for HF solution
C = rand(system.l, system.l);

In [13]:
results = zeros(system.l, n);
x = [0.314159276, 0.2718281828]
for (i, xi) in enumerate(x)
    for (j, ψ) in enumerate(system.basis.funcs)
        results[j, i] = compute(xi, ψ)
    end
end

In [14]:
results

20×2 Matrix{Float64}:
  0.524614   0.526243
  0.11654    0.10115
 -0.352652  -0.358362
 -0.140384  -0.122357
  0.289813   0.298592
  0.154355   0.135106
 -0.250563  -0.261974
 -0.163943  -0.144116
  0.221504   0.23526
  0.170969   0.150948
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0
  0.0        0.0